In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pytz import timezone
from datetime import datetime, timedelta
from skyfield import almanac, earthlib
from skyfield.api import Loader, Topos
import satpred

In [2]:
load = Loader('./data')

In [3]:
ts = load.timescale()
ephemeris = load('de421.bsp')
satellites = load.tle('http://celestrak.com/NORAD/elements/stations.txt', reload=True)

[#################################] 100% stations.txt


In [4]:
sun = ephemeris['sun']
earth = ephemeris['earth']
iss = satellites['ISS']
manhattan_beach_ca_usa = Topos(latitude='33.881519 N', longitude='118.388177 W', elevation_m=33)
pacific = timezone('US/Pacific')

In [5]:
d0 = datetime(2019, 6, 1, 0, 0)
t0 = ts.utc(pacific.localize(d0))
d1 = datetime(2019, 6, 11, 0, 0)
t1 = ts.utc(pacific.localize(d1))

In [6]:
df = pd.DataFrame(satpred.passes(t0, t1, manhattan_beach_ca_usa, iss, earth, sun, pretty=True))

In [7]:
columns = ['date', 'peak_magnitude', 'start', 'start_alt', 'start_az', 'start_d', 'culm', 'culm_alt', 
         'culm_az', 'culm_d', 'end', 'end_alt', 'end_az', 'end_d']

In [8]:
df[columns]

,date,peak_magnitude,start,start_alt,start_az,start_d,culm,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d
0,2019-06-01,0.2,22:51:29,10,NW,1497,22:52:57,23,NW,923,22:52:57,23,NW,921
1,2019-06-02,-2.1,22:02:13,10,NNW,1496,22:05:17,34,NE,706,22:05:17,34,NE,706
2,2019-06-03,-1.7,21:13:08,10,NNW,1495,21:15:44,20,NNE,1014,21:17:39,13,ENE,1298
3,2019-06-03,0.0,22:49:23,10,WNW,1495,22:50:29,18,WNW,1103,22:50:29,18,WNW,1101
4,2019-06-04,-3.4,21:59:44,10,NW,1494,22:02:55,71,WSW,441,22:02:55,71,WSW,441
5,2019-06-05,-3.4,21:10:19,10,NW,1493,21:13:37,54,NE,511,21:15:24,24,ESE,914
6,2019-06-06,-1.6,21:57:59,10,WNW,1492,22:00:37,21,SW,984,22:00:46,21,SW,986
7,2019-06-07,-2.8,21:08:04,10,WNW,1492,21:11:15,42,SW,600,21:13:23,18,SSE,1087
8,2019-06-09,-0.9,21:06:59,10,W,1489,21:08:41,13,SW,1298,21:10:24,10,SSW,1487


In [9]:
df = pd.DataFrame(satpred.passes(t0, t1, manhattan_beach_ca_usa, iss, earth, sun, visible=False, pretty=True))

In [10]:
columns = ['date', 'start', 'start_alt', 'start_az', 'start_d', 'culm', 'culm_alt', 
         'culm_az', 'culm_d', 'end', 'end_alt', 'end_az', 'end_d', 'pass_type']

In [11]:
df[columns]

,date,start,start_alt,start_az,start_d,culm,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d,pass_type
0,2019-06-01,14:43:53,10,S,1468,14:46:44,27,SE,823,14:49:36,10,ENE,1479,daylight
1,2019-06-01,16:20:37,10,W,1476,16:23:22,23,NW,909,16:26:08,10,NNE,1485,daylight
2,2019-06-01,22:51:29,10,NW,1497,22:54:49,66,NE,455,22:58:10,10,SE,1494,visible
3,2019-06-02,13:55:39,10,SSE,1468,13:57:26,14,SE,1255,13:59:15,10,E,1475,daylight
4,2019-06-02,15:30:40,10,WSW,1472,15:33:49,41,NW,604,15:36:59,10,NNE,1483,daylight
5,2019-06-02,22:02:13,10,NNW,1496,22:05:19,34,NE,706,22:08:25,10,ESE,1494,visible
6,2019-06-02,23:39:27,10,W,1495,23:41:47,18,SW,1110,23:44:07,10,S,1491,eclipsed
7,2019-06-03,14:41:02,10,SW,1469,14:44:20,84,NW,413,14:47:40,10,NE,1481,daylight
8,2019-06-03,16:20:21,10,NW,1481,16:21:27,11,NNW,1408,16:22:33,10,N,1484,daylight
9,2019-06-03,21:13:08,10,NNW,1495,21:15:44,20,NNE,1014,21:18:20,10,E,1495,visible


In [12]:
df[df['pass_type'] == 'visible'][columns]

,date,start,start_alt,start_az,start_d,culm,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d,pass_type
2,2019-06-01,22:51:29,10,NW,1497,22:54:49,66,NE,455,22:58:10,10,SE,1494,visible
5,2019-06-02,22:02:13,10,NNW,1496,22:05:19,34,NE,706,22:08:25,10,ESE,1494,visible
9,2019-06-03,21:13:08,10,NNW,1495,21:15:44,20,NNE,1014,21:18:20,10,E,1495,visible
10,2019-06-03,22:49:23,10,WNW,1495,22:52:28,34,SW,701,22:55:33,10,SSE,1491,visible
14,2019-06-04,21:59:44,10,NW,1494,22:03:04,73,SW,435,22:06:25,10,SE,1491,visible
17,2019-06-05,21:10:19,10,NW,1493,21:13:37,54,NE,511,21:16:54,10,ESE,1491,visible
22,2019-06-06,21:57:59,10,WNW,1492,22:00:37,21,SW,984,22:03:14,10,S,1489,visible
26,2019-06-07,21:08:04,10,WNW,1492,21:11:15,42,SW,600,21:14:27,10,SSE,1488,visible
34,2019-06-09,21:06:59,10,W,1489,21:08:41,13,SW,1298,21:10:24,10,SSW,1487,visible


In [13]:
above_alt_40 = df['culm_alt'] > 40. 
visible = df['pass_type'] == 'visible'
df[visible & above_alt_40][columns]

,date,start,start_alt,start_az,start_d,culm,culm_alt,culm_az,culm_d,end,end_alt,end_az,end_d,pass_type
2,2019-06-01,22:51:29,10,NW,1497,22:54:49,66,NE,455,22:58:10,10,SE,1494,visible
14,2019-06-04,21:59:44,10,NW,1494,22:03:04,73,SW,435,22:06:25,10,SE,1491,visible
17,2019-06-05,21:10:19,10,NW,1493,21:13:37,54,NE,511,21:16:54,10,ESE,1491,visible
26,2019-06-07,21:08:04,10,WNW,1492,21:11:15,42,SW,600,21:14:27,10,SSE,1488,visible


In [14]:
satpred.passes(t0, t1, manhattan_beach_ca_usa, iss, earth, sun, visible=False, pretty=False)

[{'satellite': <EarthSatellite 'ISS (ZARYA)' number=25544 epoch=2019-06-09T12:20:26Z>,
  'topos': <Topos 33deg 52' 53.5" N -118deg 23' 17.4" E>,
  'start_time': <Time tt=2458636.406285612>,
  'start_position': <Geometric position and velocity at date t center=<object object at 0x11eb28920> target=<object object at 0x10e54f1a0>>,
  'culmination_time': <Time tt=2458636.4082586467>,
  'culmination_position': <Geometric position and velocity at date t center=<object object at 0x11eb28920> target=<object object at 0x10e54f1a0>>,
  'end_time': <Time tt=2458636.4102471555>,
  'end_position': <Geometric position and velocity at date t center=<object object at 0x11eb28920> target=<object object at 0x10e54f1a0>>,
  'pass_type': 'daylight',
  'peak_magnitude': -1.6},
 {'satellite': <EarthSatellite 'ISS (ZARYA)' number=25544 epoch=2019-06-09T12:20:26Z>,
  'topos': <Topos 33deg 52' 53.5" N -118deg 23' 17.4" E>,
  'start_time': <Time tt=2458636.473458007>,
  'start_position': <Geometric position and